In [2]:
import ee
import os
import eemont
import numpy as np
import pandas as pd
from osgeo import gdal

In [3]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [4]:
def build_from_corner(feature, dx, dy, projection=None):

    coordinates = feature.geometry().coordinates()

    leftX = ee.Number(coordinates.get(0))
    leftY = ee.Number(coordinates.get(1)).subtract(dy)
    rightX = ee.Number(coordinates.get(0)).add(dx)
    rightY = ee.Number(coordinates.get(1))
    corners = ee.List([leftX, leftY, rightX, rightY])
    
    if projection is None:
        geometry = ee.Geometry.Rectangle(corners)
    else:
        geometry = ee.Geometry.Rectangle(corners, proj=projection)

    return ee.Feature(geometry)

def get_box_grid(box, nx, ny):
    coords = ee.List(box.coordinates())
    minlon = ee.Number(ee.List(ee.List(coords.get(0)).get(0)).get(0))
    maxlon = ee.Number(ee.List(ee.List(coords.get(0)).get(2)).get(0))
    minlat = ee.Number(ee.List(ee.List(coords.get(0)).get(0)).get(1))
    maxlat = ee.Number(ee.List(ee.List(coords.get(0)).get(2)).get(1))

    dx = maxlon.subtract(minlon).abs().divide(nx-1)
    dy = maxlat.subtract(minlat).abs().divide(ny-1)

    lons = ee.List.sequence(minlon, maxlon, None, nx).slice(0, -1)
    lats = ee.List.sequence(maxlat, minlat, None, ny).slice(0, -1)

    def outter_map(x):
        def inner_map(y):
            return ee.Feature(ee.Geometry.Point([ee.Number(y), ee.Number(x)]))
        return lons.map(inner_map)

    pcoords = lats.map(outter_map)

    ptsgrid = ee.FeatureCollection(pcoords.flatten())
    grid = ptsgrid.map(lambda p: build_from_corner(p, dx, dy).set('corner', p))

    return grid

In [5]:
def getLandsat(start, end, region, useMask=True, targetBands=None, sensors=None):
    if targetBands == None: 
        targetBands = ['BLUE','GREEN','RED', 'NIR','SWIR1','SWIR2']
    if sensors == None:
        sensors = dict(l4=True, l5=True, l7=True, l8=True)
    collection8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(start, end).filterBounds(region).preprocess().spectralIndices(VI_list).select(VI_list)
    collection7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
        .filterDate(start, end).filterBounds(region).preprocess().spectralIndices(VI_list).select(VI_list)
    collection5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
         .filterDate(start, end).filterBounds(region).preprocess().spectralIndices(VI_list).select(VI_list)
    col = collection5.merge(collection7).merge(collection8)
    return ee.ImageCollection(col)

def prepareL4L5(image):
    bandList = ['B1', 'B2','B3','B4','B5','B7','B6']
    scaling = [10000, 10000, 10000, 10000, 10000, 10000, 1000]
    scaled = ee.Image(image).select(bandList).divide(ee.Image.constant(scaling))
    validQA = [66, 130, 68, 132]
    mask1 = ee.Image(image).select(['pixel_qa']).remap(validQA, ee.List.repeat(1, len(validQA)), 0)
    mask2 = image.select('radsat_qa').eq(0)
    mask3 = image.select(bandList).reduce(ee.Reducer.min()).gt(0)
    mask4 = image.select("sr_atmos_opacity").unmask().lt(300)
    return ee.Image(image).addBands(scaled).updateMask(mask1.And(mask2).And(mask3).And(mask4))

def prepareL7(image):
    bandList = ['B1', 'B2','B3','B4','B5','B7','B6']
    scaling = [10000, 10000, 10000, 10000, 10000, 10000, 1000]
    scaled = ee.Image(image).select(bandList).divide(ee.Image.constant(scaling))
    validQA = [66, 130, 68, 132]
    mask1 = ee.Image(image).select(['pixel_qa']).remap(validQA, ee.List.repeat(1, len(validQA)), 0)
    mask2 = image.select('radsat_qa').eq(0)
    mask3 = image.select(bandList).reduce(ee.Reducer.min()).gt(0)
    mask4 = image.select("sr_atmos_opacity").unmask().lt(300)
    mask5 = ee.Image(image).mask().reduce(ee.Reducer.min()).focal_min(2.5)
    return ee.Image(image).addBands(scaled).updateMask(mask1.And(mask2).And(mask3).And(mask4).And(mask5))

def prepareL8(image):
    bandList = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10']
    scaling = [10000, 10000, 10000, 10000, 10000, 10000, 1000]
    validTOA = [66, 68, 72, 80, 96, 100, 130, 132, 136, 144, 160, 164]
    validQA = [322, 386, 324, 388, 836, 900]
    scaled = ee.Image(image).select(bandList).divide(ee.Image.constant(scaling))
    mask1 = ee.Image(image).select(['pixel_qa']).remap(validQA, ee.List.repeat(1, len(validQA)), 0)
    mask2 = image.select('radsat_qa').eq(0)
    mask3 = image.select(bandList).reduce(ee.Reducer.min()).gt(0)
    mask4 = ee.Image(image).select(['sr_aerosol']).remap(validTOA, ee.List.repeat(1, len(validQA)), 0)
    return ee.Image(image).addBands(scaled).updateMask(mask1.And(mask2).And(mask3).And(mask4))

def composite_by_month(imgCl, region, num_year, reducer, month_index=None):
    """ create composite based on month
        Note that the image should come from the same year!

        Parameters
        ----------
        imgCl: ee.imageCollection,
            the image collection to be composed
        region: ee.Geometry
            the region for the composition
        num_year: int
            the number of years for the composition
        reducer: ee.Reducer
            the way to reduce images within a single month
        
        Returns
        ----------
        ee.ImageCollection
            a collection containing all composed images
    """
    # determine the initial year 
    ini_year = imgCl.first().date().get('year')
    c = ee.List([])
    for y in range(num_year):
        # select images from a single year
        imgCl_subset = imgCl.filterDate(ee.Date.fromYMD(ini_year+y, 1, 1), ee.Date.fromYMD(ini_year+y+1, 1, 1))  
        # generate composite for each month
        if month_index == None:
            for i in range(12):
                # select images from a single month
                imgs = imgCl_subset.filterBounds(region).filter(ee.Filter.calendarRange(i, i+1, 'month'))
                if imgs.size().getInfo() != 0:
                    # assign the date of the last image within the interval as the 'system:time_start' to the composite
                    date_last = imgs.aggregate_array('system:time_start').reduce(ee.Reducer.last())
                    print (ee.Date(date_last).format().getInfo())
                    # assign the date appears most to the composite
                    date_mode = ee.Date(imgs.aggregate_array('system:time_start').reduce(ee.Reducer.mode())).format()
                    c = c.add(imgs.reduce(reducer)
                                    .clip(region) \
                                    .set({'system:time_start':date_last, 'date_mode': date_mode}))
        else:
            imgs = imgCl_subset.filterBounds(region).filter(ee.Filter.calendarRange(month_index[0], month_index[1], 'month'))
            if imgs.size().getInfo() != 0:
                # assign the date of the last image within the interval as the 'system:time_start' to the composite
                date_last = imgs.aggregate_array('system:time_start').reduce(ee.Reducer.last())
                print (ee.Date(date_last).format().getInfo())
                # assign the date appears most to the composite
                date_mode = ee.Date(imgs.aggregate_array('system:time_start').reduce(ee.Reducer.mode())).format()
                c = c.add(imgs.reduce(reducer)
                                .clip(region) \
                                .set({'system:time_start':date_last, 'date_mode': date_mode}))
    return ee.ImageCollection(c)

In [6]:

# composite_ls = composite_by_month(ls, study_area, 1, ee.Reducer.mean())
# ERA5 = ee.ImageCollection("ECMWF/ERA5/DAILY") \
#         .filterDate('2012-01-01', '2023-01-01',) \
#         .select(['total_precipitation'])
# composite_ERA5 = composite_by_month(ERA5, study_area, 20, ee.Reducer.sum())

### Generate landsat time series for ViT

In [12]:
global VI_list
VI_list = ['OSAVI']
study_area = ee.FeatureCollection('users/lin00370/Morocco/study_region')
gt = ee.FeatureCollection("users/lin00370/Morocco/ground_truth_rachid")
ls = getLandsat('2012-01-01', '2023-01-01', study_area, targetBands=VI_list)
region_name = ["Prefecture de Fes", 
               "Province de Moulay Yacoub",
               "Province de Sefrou",
               "Province de Taounate",
               "Province de Taza",
               "Province de Guercif"
               ]
use_region = True
for year in range(2012, 2023):
    ls = getLandsat(f'{year}-01-01', f'{year+1}-01-01', study_area, targetBands=VI_list)
    composite_ls = composite_by_month(ls, study_area, 1, ee.Reducer.mean())
    composite_ls_bands = composite_ls.toBands().rename([f"{VI_list[0]}_{m}" for m in range(1, 13)])
    if use_region: 
        for i, region in enumerate(region_name):
            aoi = study_area.filterMetadata("ADM2_EN", "equals", region)
            olive_irri = gt.filterMetadata("Irrigation", "equals", "irrigated") \
                                    .filterMetadata("Tree_type", "equals", "olive") \
                                    .filterBounds(aoi)
            olive_rf = gt.filterMetadata("Irrigation", "equals", "rainfed") \
                                    .filterMetadata("Tree_type", "equals", "olive") \
                                    .filterBounds(aoi)
            ndvi_irri = composite_ls_bands.reduceRegions(
                collection=olive_irri,
                reducer=ee.Reducer.mean(),
                scale=100)
            ndvi_rf = composite_ls_bands.reduceRegions(
                collection=olive_rf,
                reducer=ee.Reducer.mean(),
                scale=100)
        
            task_irri = ee.batch.Export.table.toDrive(
                folder=VI_list[0],
                collection=ndvi_irri,
                fileFormat="CSV",
                selectors=[f"{VI_list[0]}_1", f"{VI_list[0]}_2", f"{VI_list[0]}_3", f"{VI_list[0]}_4",
                        f"{VI_list[0]}_5", f"{VI_list[0]}_6", f"{VI_list[0]}_7", f"{VI_list[0]}_8",
                        f"{VI_list[0]}_9", f"{VI_list[0]}_10", f"{VI_list[0]}_11", f"{VI_list[0]}_12",
                        "Irrigation", "Tree_type", "FIELD_ID"],
                description=f"{region.split(' ')[-1]}_{VI_list[0]}_irri_{year}")
            task_rf = ee.batch.Export.table.toDrive(
                folder=VI_list[0],
                collection=ndvi_rf,
                fileFormat="CSV",
                selectors=[f"{VI_list[0]}_1", f"{VI_list[0]}_2", f"{VI_list[0]}_3", f"{VI_list[0]}_4",
                        f"{VI_list[0]}_5", f"{VI_list[0]}_6", f"{VI_list[0]}_7", f"{VI_list[0]}_8",
                        f"{VI_list[0]}_9", f"{VI_list[0]}_10", f"{VI_list[0]}_11", f"{VI_list[0]}_12",
                        "Irrigation", "Tree_type", "FIELD_ID"],
                description=f"{region.split(' ')[-1]}_{VI_list[0]}_rf_{year}")
            task_irri.start()
            task_rf.start()
    else:
        olive_irri = gt.filterMetadata("Irrigation", "equals", "irrigated") \
                                    .filterMetadata("Tree_type", "equals", "olive")
        olive_rf = gt.filterMetadata("Irrigation", "equals", "rainfed") \
                                .filterMetadata("Tree_type", "equals", "olive")
        ndvi_irri = composite_ls_bands.reduceRegions(
            collection=olive_irri,
            reducer=ee.Reducer.mean(),
            scale=100)
        ndvi_rf = composite_ls_bands.reduceRegions(
            collection=olive_rf,
            reducer=ee.Reducer.mean(),
            scale=100)
        task_irri = ee.batch.Export.table.toDrive(
            folder="irrigation_regional_analysis",
            collection=ndvi_irri,
            fileFormat="CSV",
            selectors=[f"{VI_list[0]}_1", f"{VI_list[0]}_2", f"{VI_list[0]}_3", f"{VI_list[0]}_4",
                    f"{VI_list[0]}_5", f"{VI_list[0]}_6", f"{VI_list[0]}_7", f"{VI_list[0]}_8",
                    f"{VI_list[0]}_9", f"{VI_list[0]}_10", f"{VI_list[0]}_11", f"{VI_list[0]}_12",
                    "Irrigation", "Tree_type", "FIELD_ID"],
            description=f"all_{VI_list[0]}_irri_{year}")
        task_rf = ee.batch.Export.table.toDrive(
            folder="irrigation_regional_analysis",
            collection=ndvi_rf,
            fileFormat="CSV",
            selectors=[f"{VI_list[0]}_1", f"{VI_list[0]}_2", f"{VI_list[0]}_3", f"{VI_list[0]}_4",
                    f"{VI_list[0]}_5", f"{VI_list[0]}_6", f"{VI_list[0]}_7", f"{VI_list[0]}_8",
                    f"{VI_list[0]}_9", f"{VI_list[0]}_10", f"{VI_list[0]}_11", f"{VI_list[0]}_12",
                    "Irrigation", "Tree_type", "FIELD_ID"],
            description=f"all_{VI_list[0]}_rf_{year}")
        task_irri.start()
        task_rf.start()


2012-01-28T10:51:35
2012-02-29T10:51:41
2012-03-16T10:51:41
2012-04-17T10:51:37
2012-05-03T10:52:04
2012-05-03T10:52:04
2012-07-06T10:51:49
2012-08-23T10:52:46
2012-09-08T10:52:57
2012-09-08T10:52:57
2012-11-20T10:47:22
2012-12-29T10:53:43
2013-01-30T10:53:46
2013-02-15T10:53:46
2013-02-15T10:53:46
2013-04-28T10:59:19
2013-05-30T10:59:37
2013-06-15T10:59:32
2013-07-17T10:59:32
2013-08-18T10:59:35
2013-09-19T10:59:30
2013-09-19T10:59:30
2013-11-06T10:59:19
2013-12-24T10:59:01
2014-01-25T10:58:41
2014-02-26T10:58:16
2014-03-30T10:57:48
2014-04-15T10:57:32
2014-05-17T10:57:05
2014-06-18T10:57:17
2014-07-20T10:57:26
2014-08-21T10:57:38
2014-09-22T10:57:40
2014-10-24T10:57:47
2014-11-25T10:57:45
2014-12-27T10:57:38
2015-01-28T10:57:31
2015-02-13T10:57:22
2015-03-17T10:57:09
2015-04-18T10:56:57
2015-05-20T10:56:36
2015-06-21T10:56:53
2015-07-23T10:57:09
2015-08-24T10:57:20
2015-09-25T10:57:34
2015-10-27T10:57:40
2015-11-28T10:57:43
2015-12-30T10:57:41
2016-01-31T10:57:37
2016-02-16T10:57:29


In [49]:
region_name = ["Prefecture de Fes", 
               "Province de Moulay Yacoub",
               "Province de Sefrou",
               "Province de Taounate",
               "Province de Taza",
            #    "Province de Guercif"
               ]
use_region = True
for year in range(2012, 2023):
    ERA5 = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") \
        .filterDate(f'{year}-01-01', f'{year+1}-01-01') \
        .select(['total_precipitation_sum'])
    composite_ERA5 = composite_by_month(ERA5, study_area, 1, ee.Reducer.sum())
    composite_ERA5_bands = composite_ERA5.toBands().rename([f"prec_{m}" for m in range(1, 13)])
    if use_region: 
        for i, region in enumerate(region_name):
            aoi = study_area.filterMetadata("ADM2_EN", "equals", region)
            olive_irri = gt.filterMetadata("Irrigation", "equals", "irrigated") \
                                    .filterMetadata("Tree_type", "equals", "olive") \
                                    .filterBounds(aoi)
            olive_rf = gt.filterMetadata("Irrigation", "equals", "rainfed") \
                                    .filterMetadata("Tree_type", "equals", "olive") \
                                    .filterBounds(aoi)
            ndvi_irri = composite_ls_bands.reduceRegions(
                collection=olive_irri,
                reducer=ee.Reducer.mean(),
                scale=100)
            ndvi_rf = composite_ls_bands.reduceRegions(
                collection=olive_rf,
                reducer=ee.Reducer.mean(),
                scale=100)
            prec = composite_ERA5_bands.reduceRegions(
                collection=olive_rf.merge(olive_irri),
                reducer=ee.Reducer.mean(),
                scale=100) 
            task_prec = ee.batch.Export.table.toDrive(
                folder="irrigation_regional_analysis",
                collection=prec,
                fileFormat="CSV",
                selectors=[f"prec_1", f"prec_2", f"prec_3", f"prec_4",
                        f"prec_5", f"prec_6", f"prec_7", f"prec_8",
                        f"prec_9", f"prec_10", f"prec_11", f"prec_12",
                        "Irrigation", "Tree_type", "FIELD_ID"],
                description=f"{region.split(' ')[-1]}_prec_{year}")
            task_prec.start()
    else:
        olive_irri = gt.filterMetadata("Irrigation", "equals", "irrigated") \
                                    .filterMetadata("Tree_type", "equals", "olive")
        olive_rf = gt.filterMetadata("Irrigation", "equals", "rainfed") \
                                .filterMetadata("Tree_type", "equals", "olive")
        ndvi_irri = composite_ls_bands.reduceRegions(
            collection=olive_irri,
            reducer=ee.Reducer.mean(),
            scale=100)
        ndvi_rf = composite_ls_bands.reduceRegions(
            collection=olive_rf,
            reducer=ee.Reducer.mean(),
            scale=100)
        prec = composite_ERA5_bands.reduceRegions(
                collection=olive_rf.merge(olive_irri),
                reducer=ee.Reducer.mean(),
                scale=100) 
        task_prec = ee.batch.Export.table.toDrive(
                folder="irrigation_regional_analysis",
                collection=prec,
                fileFormat="CSV",
                selectors=[f"prec_1", f"prec_2", f"prec_3", f"prec_4",
                        f"prec_5", f"prec_6", f"prec_7", f"prec_8",
                        f"prec_9", f"prec_10", f"prec_11", f"prec_12",
                        "Irrigation", "Tree_type", "FIELD_ID"],
                description=f"all_prec_{year}")
        task_prec.start()


2012-01-31T00:00:00
2012-02-29T00:00:00
2012-03-31T00:00:00
2012-04-30T00:00:00
2012-05-31T00:00:00
2012-06-30T00:00:00
2012-07-31T00:00:00
2012-08-31T00:00:00
2012-09-30T00:00:00
2012-10-31T00:00:00
2012-11-30T00:00:00
2012-12-31T00:00:00
2013-01-31T00:00:00
2013-02-28T00:00:00
2013-03-31T00:00:00
2013-04-30T00:00:00
2013-05-31T00:00:00
2013-06-30T00:00:00
2013-07-31T00:00:00
2013-08-31T00:00:00
2013-09-30T00:00:00
2013-10-31T00:00:00
2013-11-30T00:00:00
2013-12-31T00:00:00
2014-01-31T00:00:00
2014-02-28T00:00:00
2014-03-31T00:00:00
2014-04-30T00:00:00
2014-05-31T00:00:00
2014-06-30T00:00:00
2014-07-31T00:00:00
2014-08-31T00:00:00
2014-09-30T00:00:00
2014-10-31T00:00:00
2014-11-30T00:00:00
2014-12-31T00:00:00
2015-01-31T00:00:00
2015-02-28T00:00:00
2015-03-31T00:00:00
2015-04-30T00:00:00
2015-05-31T00:00:00
2015-06-30T00:00:00
2015-07-31T00:00:00
2015-08-31T00:00:00
2015-09-30T00:00:00
2015-10-31T00:00:00
2015-11-30T00:00:00
2015-12-31T00:00:00
2016-01-31T00:00:00
2016-02-29T00:00:00
